# Test GPU+CUDA

In [1]:
import tensorflow as tf
tf.__version__

'1.8.0'

In [2]:
a = tf.constant([1.0, 2.0, 3.0 ,4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0 ,4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4466504632788015869
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7644122317
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16777775610717725655
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:13:00.0, compute capability: 6.1"
]


# Preprocess

+ 手写字体：
    1. 上传自己的手写图片灰阶照片 
    <p align="center">
      <img src="handwriting_preparation\images\yun.jpg" alt="handwritten-pic",  width="600"/>
    </p>
    
    2. 使用Tesseract+jTessBoxEditor获取每个charactor的内容与坐标box坐标信息
    
    <p align="center">
      <img src="README.asserts/char_box.png" alt="char-box",  width="400"/>
    </p>
    
    每个字对应的box信息：
    <p align="center">
      <img src="README.asserts/box.png" alt="box-info",  width="400"/>
    </p>
+ 书法字体（target）：
    
    1. 由于名家书法作品图片不够齐全，所以使用ttf格式的计算机编码字体生成图像
    2. 对比之后，选用“迷你简黄草”字体
    <p align="center">
      <img src="README.asserts/font_viewer.png" alt="char-box",  width="400"/>
    </p>

In [4]:
!python font2img_finetune.py

['云', '551', '2363', '724', '2516', '0']
['想', '782', '2369', '1002', '2541', '0']
['衣', '1059', '2355', '1263', '2549', '0']
['裳', '1296', '2354', '1482', '2568', '0']
['花', '1539', '2367', '1718', '2561', '0']
['想', '1775', '2366', '2012', '2563', '0']
['容', '2053', '2370', '2281', '2570', '0']
['，', '2246', '2367', '2284', '2396', '0']
['春', '2368', '2362', '2572', '2575', '0']
['风', '2616', '2375', '2777', '2561', '0']
['拂', '2821', '2345', '3044', '2571', '0']
['槛', '3069', '2377', '3299', '2566', '0']
['露', '3351', '2371', '3544', '2564', '0']
['华', '3552', '2347', '3714', '2560', '0']
['浓', '3752', '2397', '3960', '2542', '0']
['。', '3921', '2367', '3956', '2397', '0']
['若', '457', '2073', '624', '2271', '0']
['非', '671', '2056', '838', '2260', '0']
['群', '862', '2039', '1059', '2273', '0']
['玉', '1080', '2119', '1249', '2237', '0']
['山', '1276', '2119', '1403', '2247', '0']
['头', '1463', '2097', '1613', '2258', '0']
['见', '1675', '2106', '1852', '2265', '0']
['，', '1894', '2114

# Train

In [5]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import print_function
from model.preprocessing_helper import CANVAS_SIZE, EMBEDDING_DIM
from model.unet import UNet


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
################ train-params ###############

experiment_dir='experiments_finetune'    # experiment directory, data, samples,checkpoints,etc
experiment_id=0         # sequence id for the experiments you prepare to run
image_size=CANVAS_SIZE  # size of your input and output image
L1_penalty=100      # weight for L1 loss
Lconst_penalty=15   # weight for const loss
Ltv_penalty=0.0     # weight for tv loss
Lcategory_penalty=1.0  # weight for tv loss
embedding_num=185   # number for distinct embeddings
embedding_dim=EMBEDDING_DIM   # dimension for embedding
epoch=10         # number of epoch
batch_size=16    # number of examples in batch
lr=0.001         # initial learning rate for adam
schedule=20         # number of epochs to half learning rate
resume=1            # number of epochs to half learning rate
resume_pre_model=0  # resume from pre-training
freeze_encoder_decoder=1      # freeze encoder/decoder weights during training
optimizer='adam'   # optimizer of the model
fine_tune='67'     # specific labels id to be fine tuned
inst_norm=1      # use conditional instance normalization in your model
sample_steps=20  # number of batches in between two samples are drawn from validation set
checkpoint_steps=50  # number of batches in between two checkpoints
validate_steps=1     # number of batches in between two validations
validate_batches=20  # validation epochs
flip_labels=1    # whether flip training data labels or not, in fine tuning

In [7]:


with tf.Session(config=config) as sess:
    model = UNet(experiment_dir=experiment_dir, batch_size=batch_size, experiment_id=experiment_id,
                 input_width=image_size, output_width=image_size, embedding_num=embedding_num,
                 validate_batches=validate_batches, embedding_dim=embedding_dim,
                 L1_penalty=L1_penalty, Lconst_penalty=Lconst_penalty,
                 Ltv_penalty=Ltv_penalty, Lcategory_penalty=Lcategory_penalty)
    model.register_session(sess)
    if flip_labels:
        model.build_model(is_training=True, inst_norm=inst_norm, no_target_source=True)
    else:
        model.build_model(is_training=True, inst_norm=inst_norm)
    fine_tune_list = None
    if fine_tune:
        ids = fine_tune.split(",")
        fine_tune_list = set([int(i) for i in ids])
#         print("@@@@@"+str(fine_tune_list))

    model.train(lr=lr, epoch=epoch, resume=resume, resume_pre_model=resume_pre_model,
                schedule=schedule, freeze_encoder_decoder=freeze_encoder_decoder,
                fine_tune=fine_tune_list,
                sample_steps=sample_steps, checkpoint_steps=checkpoint_steps,
                validate_steps=validate_steps,
                flip_labels=flip_labels, optimizer=optimizer)


freeze encoder/decoder weights
unpickled total 86 examples
unpickled total 10 examples
filter by label -> {67}
train examples -> 0, val examples -> 0

model_checkpoint_path: "experiments_finetune\\checkpoint\\experiment_0\\unet.model-0"
all_model_checkpoint_paths: "experiments_finetune\\checkpoint\\experiment_0\\unet.model-0"


INFO:tensorflow:Restoring parameters from experiments_finetune\checkpoint\experiment_0\unet.model-0
restored model experiments_finetune\checkpoint\experiment_0
Checkpoint: last checkpoint step 0


# Infer

In [8]:
# -*- coding: utf-8 -*-
import os
from model.utils import compile_frames_to_gif

In [6]:
################ infer-params ###############

experiment_dir='experiments_finetune'    # experiment directory, data, samples,checkpoints,etc
experiment_id=0         # sequence id for the experiments you prepare to run
image_size=CANVAS_SIZE  # size of your input and output image
L1_penalty=100      # weight for L1 loss
Lconst_penalty=15   # weight for const loss
Ltv_penalty=0.0     # weight for tv loss
Lcategory_penalty=1.0  # weight for tv loss
embedding_num=185   # number for distinct embeddings
embedding_dim=EMBEDDING_DIM   # dimension for embedding
epoch=10         # number of epoch
batch_size=16    # number of examples in batch
lr=0.001         # initial learning rate for adam
schedule=20         # number of epochs to half learning rate
resume=1            # number of epochs to half learning rate
resume_pre_model=0  # resume from pre-training
freeze_encoder_decoder=1      # freeze encoder/decoder weights during training
optimizer='adam'   # optimizer of the model
fine_tune='67'     # specific labels id to be fine tuned
inst_norm=1      # use conditional instance normalization in your model
sample_steps=20  # number of batches in between two samples are drawn from validation set
checkpoint_steps=50  # number of batches in between two checkpoints
validate_steps=1     # number of batches in between two validations
validate_batches=20  # validation epochs
flip_labels=1    # whether flip training data labels or not, in fine tuning

In [ ]:


parser = argparse.ArgumentParser(description='Inference for unseen data')
parser.add_argument('--model_dir', dest='model_dir', required=True,
                    help='directory that saves the model checkpoints')
parser.add_argument('--batch_size', dest='batch_size', type=int, default=16, help='number of examples in batch')
parser.add_argument('--source_obj', dest='source_obj', type=str, required=True, help='the source images for inference')
parser.add_argument('--embedding_ids', default='embedding_ids', type=str, help='embeddings involved')
parser.add_argument('--save_dir', default='save_dir', type=str, help='path to save inferred images')
parser.add_argument('--inst_norm', dest='inst_norm', type=int, default=0,
                    help='use conditional instance normalization in your model')
parser.add_argument('--interpolate', dest='interpolate', type=int, default=0,
                    help='interpolate between different embedding vectors')
parser.add_argument('--steps', dest='steps', type=int, default=10, help='interpolation steps in between vectors')
parser.add_argument('--output_gif', dest='output_gif', type=str, default=None, help='output name transition gif')
parser.add_argument('--uroboros', dest='uroboros', type=int, default=0,
                    help='Shōnen yo, you have stepped into uncharted territory')
args = parser.parse_args()



    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        model = UNet(batch_size=args.batch_size)
        model.register_session(sess)
        model.build_model(is_training=False, inst_norm=args.inst_norm)
        embedding_ids = [int(i) for i in args.embedding_ids.split(",")]
        if not args.interpolate:
            if len(embedding_ids) == 1:
                embedding_ids = embedding_ids[0]
            model.infer(model_dir=args.model_dir, source_obj=args.source_obj, embedding_ids=embedding_ids,
                        save_dir=args.save_dir)
        else:
            if len(embedding_ids) < 2:
                raise Exception("no need to interpolate yourself unless you are a narcissist")
            chains = embedding_ids[:]
            if args.uroboros:
                chains.append(chains[0])
            pairs = list()
            for i in range(len(chains) - 1):
                pairs.append((chains[i], chains[i + 1]))
            for s, e in pairs:
                model.interpolate(model_dir=args.model_dir, source_obj=args.source_obj, between=[s, e],
                                  save_dir=args.save_dir, steps=args.steps)
            if args.output_gif:
                gif_path = os.path.join(args.save_dir, args.output_gif)
                compile_frames_to_gif(args.save_dir, gif_path)
                print("gif saved at %s" % gif_path)


